In [2]:
#Imports
import json
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import openweathermapy.core as ow
from citipy import citipy 
import random
import requests as req

In [3]:
#Random selection of 500 unique cities based on lat/long coordinates
#Truncate random coordinates to 6 decimal points each
def truncate(x, d): #Source for truncate() function: https://stackoverflow.com/a/20545024
    return int(x*(10.0**d))/(10.0**d)
    #x = input
    #d = decimal places to truncate to

cityList = []
selected_cites = []
uniqueCityCount = 0

while (uniqueCityCount < 10): 
    random_lat = truncate(random.uniform(-90,90),6)
    random_long = truncate(random.uniform(-180,180),6)
    cityToCheck = citipy.nearest_city(random_lat, random_long).city_name
    #Pass query to OpenWeatherMap API to check if city is in database
    
    query_city = cityToCheck
    query_url = base_url + "?apikey=" + api_key + "&q=" + query_city
    response = req.get(query_url)
    json_data = response.json()
    
    #Prevent duplicates from being appended to the cityList (before dataframe creation)
    #While also not adding the specified city if not in OWM database
    if cityToCheck not in cityList and json_data['cod'] != '404':
        cityList.append(cityToCheck)
        uniqueCityCount = uniqueCityCount+ 1 
        
#Create dataframe with uniqueCity names        
raw_cities_df = pd.DataFrame({"City Name": cityList})

#Not needed if duplicates removed before dataframe creation: 
#raw_cities_df = raw_cities_df.drop_duplicates() 

#Creation of column for city number
raw_cities_df['City Number'] = raw_cities_df.index + 1
raw_cities_df= raw_cities_df[['City Number','City Name']]
cn = pd.to_numeric(raw_cities_df['City Number'], errors='raise', downcast=None)

In [4]:

raw_cities_df.head(25)
#print(raw_cities_df['City Number'])

,City Number,City Name
0,1,port macquarie
1,2,staraya stanitsa
2,3,bluff
3,4,nayoro
4,5,albany
5,6,langres
6,7,lerwick
7,8,port blair
8,9,port alfred
9,10,punta arenas


In [5]:
settings = {"units": "imperial","appid": api_key}
from time import sleep

#Print parameters being requested from owm API before retrival
print("Base URL for API requests:... " + ow.BASE_URL)
print("Parameters being requested per city:... ")
summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
print(summary)

#Get current weather for each city in cityList
cityCount = 1
for city in raw_cities_df["City Name"]:
    sleep(1.2)
    try:
        print("Processing city " + str(cityCount)+ " of " +str(len(raw_cities_df["City Name"])) + ", "+ city +":") 
        current_city= ow.get_current(city, **settings)
        sleep(1.2)
        #Leveraging OpenWatherMapy to parse the response with current_city(*summary)
        #summary = ["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]
        summary_output = current_city(*summary)
        print(str(summary_output))
    
    except Exception as ex:
        print(ex)
        cityCount = cityCount - 1
    cityCount = cityCount + 1

    #openweathermap complete city list:
    #http://openweathermap.org/help/city_list.txt 
    #one method ---> check against list
    #another method ---> skip if error returned 
    
    #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_html.html
    
    

Base URL for API requests:... http://api.openweathermap.org/data/2.5/
Parameters being requested per city:... 
['name', 'coord.lat', 'coord.lon', 'main.temp', 'main.humidity', 'wind.speed', 'clouds.all']
Processing city 1 of 10, port macquarie:
('Port Macquarie', -31.43, 152.91, 77, 69, 10.29, 90)
Processing city 2 of 10, staraya stanitsa:
('Staraya Stanitsa', 45.01, 41.15, 29.83, 91, 9.91, 0)
Processing city 3 of 10, bluff:
('Bluff', -23.58, 149.07, 75.41, 80, 3.87, 56)
Processing city 4 of 10, nayoro:
('Nayoro', 44.36, 142.46, 16.46, 79, 3.98, 76)
Processing city 5 of 10, albany:
('Albany', 42.65, -73.75, 44.6, 56, 11.41, 75)
Processing city 6 of 10, langres:
('Langres', 47.87, 5.33, 33.8, 94, 2.24, 88)
Processing city 7 of 10, lerwick:
('Lerwick', 60.15, -1.15, 41, 86, 23.04, 92)
Processing city 8 of 10, port blair:
('Port Blair', 11.67, 92.75, 82.07, 100, 9.01, 0)
Processing city 9 of 10, port alfred:
('Port Alfred', -33.59, 26.89, 71.54, 88, 8.9, 92)
Processing city 10 of 10, punt

In [6]:
#Put requested data into a dictionary that contains a list (intention to append to dataframe)
views = {"summary":["name", "coord.lat", "coord.lon","main.temp","main.humidity","wind.speed","clouds.all"]}
print(current_city(*views["summary"]))

#raw_cities_df[["summary"]]
#pd.DataFrame_fromt_dict
#https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.append.html
#Mapping ----> http://pandas.pydata.org/pandas-docs/version/0.19/generated/pandas.Series.map.html
#https://stackoverflow.com/questions/26666919/python-pandas-add-column-in-dataframe-from-list
#https://stackoverflow.com/questions/42632470/how-to-add-dictionaries-to-a-dataframe-as-a-row
#Documentation on OWM wrapper ---> https://github.com/crazycapivara/openweathermapy

('Punta Arenas', -53.16, -70.91, 59, 38, 20.8, 40)
